In [1]:
%reload_ext autoreload
%autoreload 2

%gui qt

import time
from tqdm import tqdm
from pathlib import Path
import numpy as np
import numba as nb
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt

import vrExperiment as vre
import vrFunctions as vrf
import basicFunctions as bf
import fileManagement as fm
import dbManagement as dbm

import redCellGUI as rgui

In [6]:
# Immediate next steps for programming work:
# - ROICaT index alignment tools

In [2]:
mouseName = 'CR_Hippocannula6'
dateString = '2022-09-05'
session = '701'
# vrExperiment registration and preprocessing 
# vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
# vrReg.processTimeline()
# vrReg.processImaging()
# vrReg.processRedCells()
# vrReg.saveParams()
# print(vrReg.sessionPath())
# del vrReg

In [2]:
mouseName = 'ATL020'
dateString = '2023-05-04'
session = '702'

In [28]:
# vrExperiment registration and preprocessing 
vrReg = vre.vrExperimentRegistration(mouseName, dateString, session, oasis=False, imaging=True)
vrReg.doPreprocessing()
vrReg.saveParams()
print(vrReg.sessionPath())
del vrReg

In session: ATL020/2023-05-04/702, vrFile['rigInfo'] does not exist. Assuming default settings for B2!
Self.value['numTrials'] set by trialInfo.trialIdx>0, but this might not be right. There might be smarter ways to determine which trials are 'good' trials...
frame2time had 2 more than suite2p output. This happens sometimes. I don't like it. I think it's because scanimage sends a TTL before starting the frame
ATL020/2023-05-04/702:spks shapes: [1233, 421, 1105, 1915, 1341]
ATL020/2023-05-04/702:roiPerPlane: [1233, 421, 1105, 1915, 1341]
ATL020/2023-05-04/702:F shapes: [1233, 421, 1105, 1915, 1341]
ATL020/2023-05-04/702:roiPerPlane: [1233, 421, 1105, 1915, 1341]
ATL020/2023-05-04/702:Fneu shapes: [1233, 421, 1105, 1915, 1341]
ATL020/2023-05-04/702:roiPerPlane: [1233, 421, 1105, 1915, 1341]
ATL020/2023-05-04/702:redcell shapes: [1233, 421, 1105, 1915, 1341]
ATL020/2023-05-04/702:roiPerPlane: [1233, 421, 1105, 1915, 1341]
ATL020/2023-05-04/702:iscell shapes: [1233, 421, 1105, 1915, 1341]


In [5]:
# Load registered vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)

In [12]:
iscell = vrexp.loadS2P('iscell', concatenate=True, checkVariables=False)
redcell = vrexp.loadS2P('redcell', concatenate=True, checkVariables=False)
np.unique(redcell[:,0])

array([0., 1.], dtype=float32)

In [8]:
%%time
speedThreshold = 5
omap, smap, lickmap, distvec = vrf.getBehaviorMaps(vrexp,speedThreshold=speedThreshold) # produce smoothed occupancy and speed maps, along with the distance bins used to create them
frameTrialIdx, framePosition, frameSpeed = vrexp.getFrameBehavior() # convert behavioral data into timeframe of spiking data
spkmap = vrf.getSpikeMap(vrexp, frameTrialIdx, framePosition, frameSpeed, distvec, omap, speedThreshold=speedThreshold, standardizeSpks=False, doSmoothing=0) # produce the spkmap 
idxFullTrials = np.all(~np.isnan(omap),axis=1) # find out which trials the mouse explored the whole environment
relmse, relcor = vrf.measureReliability(spkmap[:,idxFullTrials], numcv=3) # measure reliability of spiking
spInfo = vrf.measureSpatialInformation(omap[idxFullTrials], spkmap[:,idxFullTrials]) # measure spatial information of spiking

CPU times: total: 1.14 s
Wall time: 476 ms


In [9]:
# Code block for opening redSelectionGUI for a vrExperiment
vrexp = vre.vrExperiment(mouseName, dateString, session)
redCell = vre.redCellProcessing(vrexp)
redSelection = rgui.redSelectionGUI(redCell)

In [107]:
# fm.getCopyString('ATL027')
fm.s2pTargets('ATL027','2023-08-10','701')

\\zaru.cortexlab.net\Subjects\ATL027\2023-08-10\701
C:\Users\andrew\Documents\localData\ATL027\2023-08-10\701


In [35]:
# Database Requirements: 
# ---------------------
# GUI: db manager
# - show full database -
# - show selected entries (like for example entries with imaging but no suite2p) -
#                --> It would be sweet to make this work where there are some defaults, but then there is also a "filter command" option which you type and goes as the input to the filtering
# - click on entry and do things:
#                --> open file explorer to that session
#                --> do suite2p
#                --> do red cell management
# - update table data? 
# ---------------------
# Operational Commands: 
# - Automatically do suite2p 
# - Check if registration was done before a suite2p update
# - Find way to determine if a suite2p file was updated after the folder was made (indicating that QC was done afterwards) -- so registration should be rerun for imaging data at least
#    -- check if "iscell" is the only thing that was done after registration. In this case, just update the iscell oneData
# ---------------------
# - Add new entry GUI!!!
# ---------------------
# - Check session folders to see if new session is present that isn't registered in the database? 
# ---------------------
# vrExperimentUpdates: 
# -- if edited iscell, will need to re-register the number of ROIs per plane (in "self.values") and reprocess imaging data!

In [6]:
vrexp = vre.vrExperimentRegistration('ATL022', '2023-04-06','701')

In [7]:
vrexp.sessionPrint()

'ATL022/2023-04-06/701'

In [11]:
for p in vrexp.getSavedOne(): p.unlink()

In [15]:
with vrdb.openCursor(commitChanges=True) as cursor:
    for idx, row in vrdb.getTable().iterrows():
        cursor.execute(vrdb.createUpdateStatement('vrRegistrationDate',row['uSessionID']), None)

In [17]:
vrdb = dbm.vrDatabase()

In [21]:
vrdb.registerSessions()

In registerSessions, 'vrBehaviorVersion' is an important input that hasn't been coded yet!

Performing vrExperiment preprocessing for session: ATL012/2023-01-30/702
Cleared oneData from session: ATL012/2023-01-30/702
In session: ATL012/2023-01-30/702, vrFile['rigInfo'] does not exist. Assuming default settings for B2!
Self.value['numTrials'] set by trialInfo.trialIdx>0, but this might not be right. There might be smarter ways to determine which trials are 'good' trials...
The following npy files are present in some but not all plane folders within session ATL012/2023-01-30/702: ['Fneu_chan2', 'Fneu', 'F', 'redcell', 'iscell', 'stat', 'spks', 'F_chan2']
Each plane folder contains the following npy files: ['ops']
The following exception was raised when trying to preprocess session: ATL012/2023-01-30/702. Clearing all oneData.
Cleared oneData from session: ATL012/2023-01-30/702


<font color=red>Last traceback: [<FrameSummary file C:\Users\andrew\Documents\GitHub\vrAnalysis\vrExperiment.py, line 527 in processImaging>]</font>

<font color=red>Exception: ATL012/2023-01-30/702 is missing stat in at least one suite2p folder!</font>


Performing vrExperiment preprocessing for session: ATL012/2023-02-17/701
Cleared oneData from session: ATL012/2023-02-17/701
The following exception was raised when trying to preprocess session: ATL012/2023-02-17/701. Clearing all oneData.
Cleared oneData from session: ATL012/2023-02-17/701


<font color=red>Last traceback: [<FrameSummary file C:\Users\andrew\miniconda3\envs\vrAnalysis\lib\site-packages\scipy\io\matlab\_miobase.py, line 230 in _get_matfile_version>]</font>

...


Performing vrExperiment preprocessing for session: ATL022/2023-03-31/703
Cleared oneData from session: ATL022/2023-03-31/703
In session: ATL022/2023-03-31/703, vrFile['rigInfo'] does not exist. Assuming default settings for B2!
Self.value['numTrials'] set by trialInfo.trialIdx>0, but this might not be right. There might be smarter ways to determine which trials are 'good' trials...
Computing red cell features for ATL022/2023-03-31/703... (usually takes 10-20 seconds)
Facecam preprocessing has not been coded yet!
Saving params...
Accumulated oneData registered: 1.315 GB. Averaging: 1.315 GB / session. Estimate total: 5.259 GB

Performing vrExperiment preprocessing for session: ATL022/2023-04-04/701
Cleared oneData from session: ATL022/2023-04-04/701
In session: ATL022/2023-04-04/701, vrFile['rigInfo'] does not exist. Assuming default settings for B2!
Self.value['numTrials'] set by trialInfo.trialIdx>0, but this might not be right. There might be smarter ways to determine which trials a

<font color=red>Last traceback: [<FrameSummary file C:\Users\andrew\miniconda3\envs\vrAnalysis\lib\site-packages\numpy\lib\format.py, line 831 in read_array>]</font>

<font color=red>Exception: cannot reshape array of size 0 into shape (3278,16867)</font>


Performing vrExperiment preprocessing for session: ATL022/2023-04-06/701
Cleared oneData from session: ATL022/2023-04-06/701
In session: ATL022/2023-04-06/701, vrFile['rigInfo'] does not exist. Assuming default settings for B2!
Self.value['numTrials'] set by trialInfo.trialIdx>0, but this might not be right. There might be smarter ways to determine which trials are 'good' trials...
Computing red cell features for ATL022/2023-04-06/701... (usually takes 10-20 seconds)
Facecam preprocessing has not been coded yet!
Saving params...
Accumulated oneData registered: 4.931 GB. Averaging: 1.644 GB / session. Estimate total: 1.644 GB

Performing vrExperiment preprocessing for session: ATL022/2023-04-11/701
Cleared oneData from session: ATL022/2023-04-11/701
In session: ATL022/2023-04-11/701, vrFile['rigInfo'] does not exist. Assuming default settings for B2!
Self.value['numTrials'] set by trialInfo.trialIdx>0, but this might not be right. There might be smarter ways to determine which trials a

ValueError: math domain error

In [1]:
import fileManagement as fm
fm.s2pTargets('ATL022','2023-04-21','701')

\\zaru.cortexlab.net\Subjects\ATL022\2023-04-21\701
C:\Users\andrew\Documents\localData\ATL022\2023-04-21\701


In [23]:
len(df)

89

In [19]:
# vrdb.updateSuite2pDateTime()
# vrdb.checkS2P(withDatabaseUpdate=True)

Database said suite2p didn't run, but it already did: ATL020/2023-03-31/701


In [17]:
vrdb.printRequiresS2P(needsQC=True)

Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-18/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-20/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-20/702
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-23/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-24/703
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-25/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-26/702
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-27/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-01-30/702
Database indicates that suite2p has been run but not QC'd: ATL012/2023-02-01/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-02-02/701
Database indicates that suite2p has been run but not QC'd: ATL012/2023-02-06/701
Database indicates that suit

In [3]:
import PyQt5

In [6]:
dir(PyQt5)

['QtCore',
 'QtGui',
 'QtSvg',
 'QtWidgets',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'sip']